In [ ]:
import astropy.units as u
import numpy as np
import gala.potential as gp
from gala.units import dimensionless

import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline

from scipy.misc import derivative
from scipy.integrate import quad
from scipy.interpolate import interp1d
from scipy.optimize import root

In [ ]:
pot = gp.HernquistPotential(m=1., c=1., units=dimensionless)

In [ ]:
dm_dr = np.zeros(128)
r0s = np.logspace(-2, 1., dm_dr.size)
for i,r0 in enumerate(r0s):
    d = derivative(lambda r: pot.c_instance.mass_enclosed(np.array([[r,0,0]]), 1.)[0], r0, dx=1E-3*r0)
    dm_dr[i] = d

In [ ]:
plt.loglog(r0s, dm_dr/(4*np.pi*r0s**2))

q = np.zeros((3,r0s.size))
q[0] = r0s
plt.loglog(r0s, pot.density(q))

---

In [ ]:
from scipy.special import gammainc

In [ ]:
rmax = 100.
n_s = 2.2 
R_e = 4.

an = 2 * n_s
bn = 2. * n_s - 1./3. + 0.0098765/n_s + 0.0018/n_s**2 # approximation from Ciotti ...something

argc = bn * (rmax/R_e) ** (1./n_s)
gamnorm = gammainc(an, argc)

def _sersic_mass_enclosed(r):
    arg = bn * (r/R_e)**(1./n_s)
    return gammainc(an, arg) / gamnorm

In [ ]:
dm_dr = np.zeros(4096)
r0s = np.logspace(-2, 2., dm_dr.size)
for i,r0 in enumerate(r0s):
    d = derivative(_sersic_mass_enclosed, r0, dx=1E-3*r0)
    dm_dr[i] = d

In [ ]:
plt.loglog(r0s, dm_dr/(4*np.pi*r0s**2))

_hernquist = gp.HernquistPotential(m=1., c=2., units=dimensionless)
q = np.zeros((3,r0s.size))
q[0] = r0s
plt.loglog(r0s, _hernquist.density(q))

In [ ]:
from scipy.special import gamma, gammainc
def _sersic_frac_mass_enclosed(r, rmax=np.inf, n_s=2.2, R_e=4.):
    """
    This was adapted from Oleg's C code (uncluster/src/gc_evolution.c).
    For the Milky Way, the parameters (r_s, n_s, etc.) don't evolve and z=0.
    """
    a_n = 2 * n_s
    b_n = 2. * n_s - 1./3. + 0.0098765/n_s + 0.0018/n_s**2 # approximation from Ciotti ...something
    
    argc = b_n * (rmax/R_e) ** (1./n_s)
    gamnorm = gammainc(a_n, argc)
    
    arg = b_n * (r/R_e)**(1./n_s)
    return gammainc(a_n, arg) / gamnorm

In [ ]:
_sersic_frac_mass_enclosed(1000., rmax=np.inf)